# 2018-06-08 - Comparaison modèle/humain

# Le convo

In [2]:
import torch
import torchvision
from torchvision import transforms, datasets

data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1, drop_last = True)

#test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=True,
                                             num_workers=1, drop_last = True)

In [3]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 20)
        self.pool = nn.MaxPool2d(2,2)

        self.fc3 = nn.Linear(17496,1000)
        self.dropout = nn.Dropout(0.2)

        self.outlayer = nn.Linear(1000,16)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))

        x = x.view(x.size(0), -1) #reshape from conv to linear

        x = F.leaky_relu(self.fc3(x))
        x = self.dropout(x)

        x = self.outlayer(x)
        return x
        
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(20, 20), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc3): Linear(in_features=17496, out_features=1000, bias=True)
  (dropout): Dropout(p=0.2)
  (outlayer): Linear(in_features=1000, out_features=16, bias=True)
)


In [4]:
#freeze layers after and including freezing_layer+1 (layers start at 0)
def freeze_layers(freezing_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if count < freezing_layer+1 : #to freeze at iteration 1
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#freeze all the layers except the unfrozen one
def freeze_all_layers(unfrozen_layer, nn_model = model) :
    for count,child in enumerate(model.children()) :
        if count == unfrozen_layer :
            print("Layer no. %s -- %s -- NOT FROZEN"% (count,child))
            for param in child.parameters():
                param.requires_grad = True
        else :
            print("Layer no. %s -- %s -- FROZEN"%(count, child))
            for param in child.parameters():
                param.requires_grad = False

#unfreeze every layers
def layers_microwave(nn_model = model) :
    for count,child in enumerate(model.children()) :
        for param in child.parameters():
            param.requires_grad = True

In [5]:
import torch.optim as optim
import time

#unfreeze before starting
layers_microwave(model)

criterion = nn.CrossEntropyLoss() #loss criterion
optimizer = optim.SGD(params = model.parameters(),lr=0.001, momentum=0.9)
epochs = 2 #nbr of epochs per layer
model_size = 4 #nbr of layers

print_interval = 50 #prints every p_i*4
tempo = []
acc = []

start_time = time.time()
print("Started training")

for epoch in range(epochs):  # nbr epochs
    for batch_idx, (data, target) in enumerate(train_loader): #nbr batch,in,out
        data, target = Variable(data), Variable(target)
        #On resize pour la sortie


        #init l'entrainement
        optimizer.zero_grad()
        net_out = model(data)

        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        #afficher la progression
        if batch_idx % print_interval == 0:
            #le print statement le plus illisible du monde
            print('Epoch: {} [{}/{} ({:.0f}%)]\t\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])


print("Finished training in  %.3f seconds " % (time.time() - start_time))

Started training


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 1 [0/3840 (0%)]		Loss: 2.780261
Epoch: 1 [200/3840 (5%)]		Loss: 2.730807
Epoch: 1 [400/3840 (10%)]		Loss: 2.804087
Epoch: 1 [600/3840 (16%)]		Loss: 2.440760
Epoch: 1 [800/3840 (21%)]		Loss: 2.471992
Epoch: 1 [1000/3840 (26%)]		Loss: 2.250546
Epoch: 1 [1200/3840 (31%)]		Loss: 2.128938
Epoch: 1 [1400/3840 (36%)]		Loss: 2.084347
Epoch: 1 [1600/3840 (42%)]		Loss: 1.673000
Epoch: 1 [1800/3840 (47%)]		Loss: 1.167743
Epoch: 1 [2000/3840 (52%)]		Loss: 1.208961
Epoch: 1 [2200/3840 (57%)]		Loss: 0.485643
Epoch: 1 [2400/3840 (62%)]		Loss: 0.092124
Epoch: 1 [2600/3840 (68%)]		Loss: 0.139440
Epoch: 1 [2800/3840 (73%)]		Loss: 0.060365
Epoch: 1 [3000/3840 (78%)]		Loss: 0.060049
Epoch: 1 [3200/3840 (83%)]		Loss: 0.039726
Epoch: 1 [3400/3840 (89%)]		Loss: 0.169030
Epoch: 1 [3600/3840 (94%)]		Loss: 0.023138
Epoch: 1 [3800/3840 (99%)]		Loss: 0.004795


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 2 [0/3840 (0%)]		Loss: 0.003888
Epoch: 2 [200/3840 (5%)]		Loss: 0.103883
Epoch: 2 [400/3840 (10%)]		Loss: 0.028026
Epoch: 2 [600/3840 (16%)]		Loss: 0.435186
Epoch: 2 [800/3840 (21%)]		Loss: 0.006930
Epoch: 2 [1000/3840 (26%)]		Loss: 0.036822
Epoch: 2 [1200/3840 (31%)]		Loss: 0.035420
Epoch: 2 [1400/3840 (36%)]		Loss: 0.020118
Epoch: 2 [1600/3840 (42%)]		Loss: 0.081812
Epoch: 2 [1800/3840 (47%)]		Loss: 0.044806
Epoch: 2 [2000/3840 (52%)]		Loss: 0.058420
Epoch: 2 [2200/3840 (57%)]		Loss: 0.027945
Epoch: 2 [2400/3840 (62%)]		Loss: 0.045221
Epoch: 2 [2600/3840 (68%)]		Loss: 0.024116
Epoch: 2 [2800/3840 (73%)]		Loss: 0.185364
Epoch: 2 [3000/3840 (78%)]		Loss: 0.002476
Epoch: 2 [3200/3840 (83%)]		Loss: 0.015055
Epoch: 2 [3400/3840 (89%)]		Loss: 0.008820
Epoch: 2 [3600/3840 (94%)]		Loss: 0.003752
Epoch: 2 [3800/3840 (99%)]		Loss: 0.009988
Finished training in  1073.382 seconds 


In [6]:
# delete
test_loss = 0
correct = 0
model = model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)

    net_out = model(data)
    
    #somme des pertes du batch
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1] #prediction
    correct += pred.eq(target.data).sum() #output du réseau

test_loss /= len(test_loader.dataset) #loss = loss/length set
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # This is added back by InteractiveShellApp.init_path()



Test set: Average loss: 0.0289, Accuracy: 540/576 (93%)



## Convo - Psychophysique

In [7]:
exp_info = 'convo_model_HIRES'
print (exp_info)

convo_model_HIRES


On défini la génération de MotionClouds :

In [8]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [10]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
from PIL import Image

test_length = 600
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()


ans_list = []
N_B_thetas = 15
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2)
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])


At trial  0 Angle= -25.909 answer is  tensor(9) (correct= False ); bandwidth= 33.435
At trial  1 Angle= 3.038 answer is  tensor(7) (correct= False ); bandwidth= 15.910
At trial  2 Angle= -45.000 answer is  tensor(11) (correct= False ); bandwidth= 15.910
At trial  3 Angle= -4.402 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  4 Angle= -4.807 answer is  tensor(8) (correct= True ); bandwidth= 2.812
At trial  5 Angle= -19.932 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  6 Angle= -45.000 answer is  tensor(11) (correct= False ); bandwidth= 5.911
At trial  7 Angle= -10.806 answer is  tensor(11) (correct= False ); bandwidth= 33.435
At trial  8 Angle= 10.082 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  9 Angle= -11.689 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  10 Angle= -2.747 answer is  tensor(8) (correct= True ); bandwidth= 12.421
At trial  11 Angle= 4.995 answer is  tensor(15) (correct= False ); 

At trial  97 Angle= 45.000 answer is  tensor(4) (correct= False ); bandwidth= 5.911
At trial  98 Angle= -28.076 answer is  tensor(10) (correct= False ); bandwidth= 12.421
At trial  99 Angle= -3.205 answer is  tensor(8) (correct= True ); bandwidth= 20.379
At trial  100 Angle= 34.094 answer is  tensor(5) (correct= False ); bandwidth= 5.911
At trial  101 Angle= -34.121 answer is  tensor(14) (correct= False ); bandwidth= 42.826
At trial  102 Angle= 20.712 answer is  tensor(6) (correct= False ); bandwidth= 3.602
At trial  103 Angle= 37.511 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  104 Angle= 40.994 answer is  tensor(4) (correct= False ); bandwidth= 20.379
At trial  105 Angle= -30.076 answer is  tensor(11) (correct= False ); bandwidth= 42.826
At trial  106 Angle= 5.446 answer is  tensor(7) (correct= False ); bandwidth= 4.614
At trial  107 Angle= 24.202 answer is  tensor(6) (correct= False ); bandwidth= 20.379
At trial  108 Angle= -10.511 answer is  tensor(8) (corre

At trial  193 Angle= 7.959 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  194 Angle= -34.057 answer is  tensor(10) (correct= False ); bandwidth= 7.571
At trial  195 Angle= 6.794 answer is  tensor(7) (correct= False ); bandwidth= 4.614
At trial  196 Angle= -45.000 answer is  tensor(14) (correct= False ); bandwidth= 90.000
At trial  197 Angle= 34.421 answer is  tensor(5) (correct= False ); bandwidth= 2.812
At trial  198 Angle= 45.000 answer is  tensor(4) (correct= False ); bandwidth= 5.911
At trial  199 Angle= 15.596 answer is  tensor(6) (correct= False ); bandwidth= 26.103
At trial  200 Angle= 25.709 answer is  tensor(5) (correct= False ); bandwidth= 12.421
At trial  201 Angle= -45.000 answer is  tensor(11) (correct= False ); bandwidth= 26.103
At trial  202 Angle= -20.366 answer is  tensor(9) (correct= False ); bandwidth= 5.911
At trial  203 Angle= 23.259 answer is  tensor(15) (correct= False ); bandwidth= 33.435
At trial  204 Angle= -9.238 answer is  tensor(15) (c

At trial  289 Angle= 33.287 answer is  tensor(5) (correct= False ); bandwidth= 9.697
At trial  290 Angle= 8.516 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  291 Angle= 45.000 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  292 Angle= 20.707 answer is  tensor(6) (correct= False ); bandwidth= 20.379
At trial  293 Angle= 43.184 answer is  tensor(15) (correct= False ); bandwidth= 90.000
At trial  294 Angle= -18.789 answer is  tensor(10) (correct= False ); bandwidth= 26.103
At trial  295 Angle= -34.890 answer is  tensor(10) (correct= False ); bandwidth= 7.571
At trial  296 Angle= 23.510 answer is  tensor(5) (correct= False ); bandwidth= 2.812
At trial  297 Angle= -32.429 answer is  tensor(10) (correct= False ); bandwidth= 4.614
At trial  298 Angle= -9.928 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  299 Angle= 44.521 answer is  tensor(4) (correct= False ); bandwidth= 4.614
At trial  300 Angle= -9.124 answer is  tensor(15) 

At trial  385 Angle= -13.301 answer is  tensor(9) (correct= False ); bandwidth= 7.571
At trial  386 Angle= 22.613 answer is  tensor(15) (correct= False ); bandwidth= 42.826
At trial  387 Angle= -23.280 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  388 Angle= 11.205 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  389 Angle= 10.498 answer is  tensor(7) (correct= False ); bandwidth= 3.602
At trial  390 Angle= 28.367 answer is  tensor(6) (correct= False ); bandwidth= 33.435
At trial  391 Angle= 32.653 answer is  tensor(5) (correct= False ); bandwidth= 15.910
At trial  392 Angle= -20.919 answer is  tensor(9) (correct= False ); bandwidth= 3.602
At trial  393 Angle= -11.964 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  394 Angle= -25.604 answer is  tensor(10) (correct= False ); bandwidth= 26.103
At trial  395 Angle= -43.574 answer is  tensor(11) (correct= False ); bandwidth= 20.379
At trial  396 Angle= 31.403 answer is  tensor

At trial  481 Angle= -23.284 answer is  tensor(9) (correct= False ); bandwidth= 2.812
At trial  482 Angle= -16.583 answer is  tensor(15) (correct= False ); bandwidth= 90.000
At trial  483 Angle= 35.334 answer is  tensor(15) (correct= False ); bandwidth= 70.264
At trial  484 Angle= 13.801 answer is  tensor(6) (correct= False ); bandwidth= 2.812
At trial  485 Angle= -0.834 answer is  tensor(7) (correct= True ); bandwidth= 20.379
At trial  486 Angle= -7.037 answer is  tensor(15) (correct= False ); bandwidth= 42.826
At trial  487 Angle= 8.481 answer is  tensor(7) (correct= False ); bandwidth= 2.812
At trial  488 Angle= 0.424 answer is  tensor(8) (correct= False ); bandwidth= 15.910
At trial  489 Angle= -7.264 answer is  tensor(15) (correct= False ); bandwidth= 54.856
At trial  490 Angle= -45.000 answer is  tensor(11) (correct= False ); bandwidth= 15.910
At trial  491 Angle= -11.769 answer is  tensor(9) (correct= False ); bandwidth= 12.421
At trial  492 Angle= -45.000 answer is  tensor(11) 

At trial  577 Angle= -9.915 answer is  tensor(8) (correct= True ); bandwidth= 12.421
At trial  578 Angle= -16.550 answer is  tensor(9) (correct= False ); bandwidth= 12.421
At trial  579 Angle= 34.087 answer is  tensor(5) (correct= False ); bandwidth= 15.910
At trial  580 Angle= 38.963 answer is  tensor(4) (correct= False ); bandwidth= 15.910
At trial  581 Angle= 20.860 answer is  tensor(6) (correct= False ); bandwidth= 20.379
At trial  582 Angle= -3.250 answer is  tensor(15) (correct= False ); bandwidth= 90.000
At trial  583 Angle= -18.575 answer is  tensor(7) (correct= True ); bandwidth= 42.826
At trial  584 Angle= 5.525 answer is  tensor(6) (correct= False ); bandwidth= 42.826
At trial  585 Angle= 33.795 answer is  tensor(5) (correct= False ); bandwidth= 26.103
At trial  586 Angle= -18.836 answer is  tensor(9) (correct= False ); bandwidth= 5.911
At trial  587 Angle= -35.379 answer is  tensor(10) (correct= False ); bandwidth= 15.910
At trial  588 Angle= 10.637 answer is  tensor(15) (c

In [12]:
import pickle
pickle.dump(ans_list, open('./psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Le Ring

In [14]:
"""
Created on Tue May 15 11:25:41 2018

@author: hugo
"""
from torch.autograd import Variable
import torch 
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F

#Transform
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((128,128)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

#Train
train_set = datasets.ImageFolder(root='16_clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=8, shuffle=True,
                                             num_workers=1)

#Test
test_set = datasets.ImageFolder(root='16_clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=8,shuffle=False,
                                             num_workers=1)

###################################################################################

# Hyper-parameters
sequence_length = 49
input_size = 49
hidden_size = 128
num_layers = 2
num_classes = 16
batch_size = 8
num_epochs = 60
learning_rate = 0.003

###################################################################################

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout = 0.3)
        self.conv1 = nn.Conv2d(1,8,30)
        self.conv2 = nn.Conv2d(1,8,30)
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(2,2)
        #self.drop = nn.Dropout(0.2)

        self.fc = nn.Linear(113 , 16)  # 2 for bidirection
    
    def forward(self, x):
        #print('Input %s' % str(x.size()))

        out = self.pool1(F.relu(self.conv1(x)))
        #print('Convoluted %s' % str(x.size()))

        # Init
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        h0 = Variable(h0).cuda()
        c0 = Variable(c0).cuda()
        
        out = out[:,-1,:,:]
        #print('Resized for RNN %s' % str(x.size()))
        
        # LSTM forward
        out, _ = self.rnn(out, (h0,c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        #print('After RNN %s' % str(out.size()))
        
        out = out.unsqueeze(1)
        #print(out.size())
        out = self.pool2(F.relu(self.conv2(out)))
        #print(out.size())
        out = out[:,-1,:,:]
        #Dropout
        #out = self.drop(out)
        #print(out.size())
        
        out = out[:,-1,:]
        # LSTM output
        out = self.fc(out)
        #print('Reshaped for output %s \n'%  str(out.size()))

        return out

model = BiRNN(input_size, hidden_size, num_layers, num_classes).cuda()
print(model)

###################################################################################

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

###################################################################################

#plotting list
loss_list = []
time_list = []
t = 0

print("Start training")
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.data[0]))
            loss_list.append(loss.data[0])
            time_list.append(t)
            t+=1

correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += predicted.eq(labels.data).sum()

print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(time_list, loss_list)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss for LSTM-CNN')
plt.show()
            

BiRNN(
  (rnn): LSTM(49, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (conv1): Conv2d(1, 8, kernel_size=(30, 30), stride=(1, 1))
  (conv2): Conv2d(1, 8, kernel_size=(30, 30), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=113, out_features=16, bias=True)
)
Start training


Process Process-4:
Traceback (most recent call last):
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hugo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/hugo/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Exception ignored in: <boun

KeyboardInterrupt: 

In [ ]:
#NOW FOR THE PSYCHOPHYSICS
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

exp_info = 'model_ring_HIRES'
print (exp_info)


downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = './tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

In [ ]:
from psychopy import visual, core, event
import MotionClouds as MC
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
from PIL import Image

test_length = 600
MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
model = model.eval()


ans_list = []
N_B_thetas = 15
B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
    
std_theta = np.pi/6

for trial in range(test_length):
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2

    varimgmc2 = data_transform(Image.open(MC2))
    #varimgmc2 = data_transform(Image.open('16_clouds_easy/0.0/B0 1.40625.png'))
    varimgmc2 = Variable(varimgmc2)
    varimgmc2 = varimgmc2.unsqueeze(0)
    
    net_ans = model(varimgmc2)
    pred = net_ans.data.max(1)[1] #prediction
    #print(pred)
    #correct = (np.sign(theta) > 0) and (pred[0]>8)
    if np.sign(theta) > 0 and pred[0] > 8 :
        correct = True
    if np.sign(theta) < 0 and pred[0] <= 8:
        correct = True
    else :
        correct = False
    print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', pred[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    if pred[0] >8 :
        ans_list.append([trial, theta, 'left', B_theta])
    if pred[0] <= 8 :
        ans_list.append([trial, theta, 'right', B_theta])


In [ ]:
import pickle
pickle.dump(ans_list, open('../psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))

# Trash

In [52]:
varimgmc2 = data_transform(Image.open('16_clouds_easy/2.722713633111154/B0 10.928413651643268.png'))
varimgmc2 = Variable(varimgmc2)
varimgmc2 = varimgmc2.unsqueeze(0)

net_ans = model(varimgmc2)
print(net_ans.data.max(1)[1])


 13
[torch.LongTensor of size 1]

